# DAT405 Introduction to Data Science and AI 
## 2022-2023, Reading Period 3
## Assignment 4: Spam classification using Naïve Bayes 
This assignmetn has three obligatory questions which will be grades as PASS/FAIL. Questions 4-5 are optional and will not be graded, but can be interesting for students aiming for higher grades.

The exercise takes place in this notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 
7zip (https://www.7-zip.org/download.html) to decompress the data.

**What to submit:** 
Convert the notebook to a pdf-file and submit it. Make sure all cells are executed so all your code and its results are included. Double check the pdf displays correctly before you submit it.

In [16]:
#Download and extract data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '20021010_easy_ham.tar.bz2'
tar: Error opening archive: Failed to open '20021010_hard_ham.tar.bz2'
tar: Error opening archive: Failed to open '20021010_spam.tar.bz2'


*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [17]:
!ls -lah

'ls' is not recognized as an internal or external command,
operable program or batch file.


### 1. Preprocessing: 
Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text (in the optional part further down can experiment with filtering out the headers and footers). 
1.	We don’t want to train and test on the same data (it might help to reflect on why if you don't recall). Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`). Use easy_ham for quesions 1 and 2.


In [81]:
# Write your pre-processing code here

import pandas as pd
import os
import sklearn.model_selection as skm

# Read data from folder and sort it into a dataframe
def readData(folder):
    files = os.listdir(folder) # 'easy_ham'
    df = pd.DataFrame(columns=['file_name', 'data'])

    for i, file in enumerate(files):
        with open(folder + '/' + file, 'r', encoding="latin-1") as f:
            temp_df = pd.DataFrame({'file_name': file, 'data': f.read()}, index=[i])
            df = pd.concat([df, temp_df])
    return df

# Read data
spam = readData('spam')
ham = readData('easy_ham')

# Create X and y
spam_X = spam['data']
spam_y = spam['file_name']

ham_X = ham['data']
ham_y = ham['file_name']

# Split data into training and testing
s_X_train, s_X_test, s_y_train, s_y_test = skm.train_test_split(spam_X, spam_y, test_size=0.5)
h_X_train, h_X_test, h_y_train, h_y_test = skm.train_test_split(ham_X, ham_y, test_size=0.5)

### 2. Write a Python program that: 
1.	Uses the four datasets from Qustion 1 (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (use the [scikit-learn library](https://scikit-learn.org/stable/)) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` ([Documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)) to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in scikit-learn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem:
- Multinomial Naive Bayes  
- Bernoulli Naive Bayes. 

Please inspect the documentation to ensure input to the classifiers is appropriate before you start coding. 



In [82]:
#Write your code here

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score


# Create a CountVectorizer object
count_vect = CountVectorizer(analyzer="word")

# Fit and transform the training data
s_X_train_counts = count_vect.fit_transform(s_X_train)
h_X_train_counts = count_vect.fit_transform(h_X_train)

# Fit and transform the testing data
s_X_test_counts = count_vect.transform(s_X_test)
h_X_test_counts = count_vect.transform(h_X_test)

# ---------------------------------------------------------------

print(len(s_X_train_counts.toarray()))
print(len(s_X_test_counts.toarray()))
print(len(h_X_train_counts.toarray()))
print(len(h_X_test_counts.toarray()))

# ---------------------------------------------------------------

# Fit the training data
s_model_Multi = MultinomialNB().fit(s_X_train_counts, s_y_train)
h_model_Multi = MultinomialNB().fit(h_X_train_counts, h_y_train)

# Predict the testing data
s_y_pred_Multi = s_model_Multi.predict(s_X_test_counts)
h_y_pred_Multi = s_model_Multi.predict(h_X_test_counts)

# ---------------------------------------------------------------

# Fit the training data
s_model_Bern = BernoulliNB().fit(s_X_train_counts, s_y_train)
h_model_Bern = BernoulliNB().fit(h_X_train_counts, h_y_train)

# Predict the testing data
s_y_pred_Bern = s_model_Bern.predict(s_X_test_counts)
h_y_pred_Bern = s_model_Bern.predict(h_X_test_counts)

# ---------------------------------------------------------------

# Compute the accuracy of the two models
print("Multinomial")
print("Spam Accuracy: ", accuracy_score(s_y_test, s_y_pred_Multi))
print("Ham Accuracy: ", accuracy_score(h_y_test, h_y_pred_Multi))

print("-"*25)

print("Bernoulli")
print("Spam Accuracy: ", accuracy_score(s_y_test, s_y_pred_Bern))
print("Ham Accuracy: ", accuracy_score(h_y_test, h_y_pred_Bern))






245
256
1249
1302


ValueError: X has 31025 features, but MultinomialNB is expecting 26243 features as input.

### 3.Run on hard ham:
Run the two models from Question 2 on spam versus hard-ham and compare to easy-ham results.

In [ ]:
#Code to report results here

### 4.	OPTIONAL - NOT MARKED: 
To avoid classification based on common and uninformative words it is common to filter these out. 

**a.** Think about why this may be useful. Show a few examples of too common and too uncommon words. 

**b.** Use the parameters in scikit-learn’s `CountVectorizer` to filter out these words. Update the program from point 2 and run it on easy ham vs spam and hard ham vs spam and report your results.

In [ ]:
#Write your code here

### 5. OPTIONAL - NOT MARKED: Eeking out further performance
Filter out the headers and footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
- Does the result improve from 3 and 4? 
- What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages or vice versa? 
- Look at the `fit_prior` parameter. What does this parameter mean? Discuss in what settings it can be helpful (you can also test your hypothesis). 

In [ ]:
#Write your code here